<a href="https://colab.research.google.com/github/Shufen-Yin/Artificial-Intelligence/blob/main/Assignment_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import Libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [3]:
# Task 1 Dataset Preparation:
# Dataset Choice
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

# Load a sample text (e.g., Jane Austen's Emma)
text = gutenberg.raw('austen-emma.txt')

# Check first 500 characters
print(text[:500])


# Load a sample text (e.g., Jane Austen)
text = gutenberg.raw('austen-emma.txt')
print(text[:1000])


[nltk_data] Downloading package gutenberg to /root/nltk_data...


[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t
[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died 

[nltk_data]   Unzipping corpora/gutenberg.zip.


In [14]:
#2 Text Preprocessing
import nltk,re

# Basic text cleaning
text = text.lower()
text = re.sub(r'[^a-z\s]', '', text)

# Example: load text (this is just an example with gutenberg)
import nltk, re
nltk.download('gutenberg')
from nltk.corpus import gutenberg

text = gutenberg.raw('austen-emma.txt')

# cleanup
text = text.lower()
text = re.sub(r'[^a-z\s]', ' ', text)


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [15]:
# Dataset Visualization / Stats
from collections import Counter
words = text.split()
print("Total words:", len(words))
print("Unique words:", len(set(words)))
print("Sample:", words[:50])


Total words: 161975
Unique words: 7096
Sample: ['emma', 'by', 'jane', 'austen', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', 'handsome', 'clever', 'and', 'rich', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', 'and', 'had', 'lived', 'nearly', 'twenty', 'one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', 'she']


**Task 2 Exploring GPTs: Model Architecture**

# 1 Transformer Model Overview

GPT (Generative Pre-trained Transformer) is based on the Transformer architecture (Vaswani et al., 2017), which replaced RNNs for sequence modeling.

Key features:

Self-Attention Mechanism: Each token in the input sequence “attends” to other tokens, capturing context efficiently.

Stacked Transformer Blocks: Each block has:

Multi-head self-attention

Feed-forward neural network

Layer normalization and residual connections

Decoder-only Architecture (GPT): GPT uses only the transformer decoder stack for causal language modeling, meaning it predicts the next token based on previous tokens.

# Text Generation Process

Tokenization: Converts text into tokens (words, subwords, or characters) using methods like Byte Pair Encoding (BPE).

Probability Distribution: GPT outputs a probability for each token in the vocabulary for the next position.

Sequence Generation:

Start with a seed text (prompt).

Iteratively sample the next token (using greedy, top-k, or nucleus sampling).

Append it to the sequence and repeat until desired length is reached.

Key idea: GPT predicts one token at a time using contextual embeddings from previous tokens.

In [18]:
# 2  Training
# Implementing a Basic Text Generation Model
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Prepare corpus
corpus = text.split()  # split into words
print("Total words in corpus:", len(corpus))

# 2. Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print("Vocabulary size:", total_words)




Total words in corpus: 161975
Vocabulary size: 7097


In [20]:
sequences_ids = tokenizer.texts_to_sequences([" ".join(corpus)])[0]
# use only first 10000 tokens to build sequences
small_ids = sequences_ids[:10000]
max_len = 30

input_sequences = []
for i in range(max_len, len(small_ids)):
    input_sequences.append(small_ids[i-max_len:i+1])

print("Number sequences (small):", len(input_sequences))


Number sequences (small): 9970


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# If sequences are equal length
max_seq_len = max(len(s) for s in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
print("input shape:", input_sequences.shape, "X shape:", X.shape, "y shape:", y.shape)


input shape: (9970, 31) X shape: (9970, 30) y shape: (9970,)


In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

max_seq_len = max(len(s) for s in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')


X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (9970, 30)
y shape: (9970,)


In [27]:
# Reduce dataset size for faster training
N = 8000   # you can try 2000 / 5000 / 8000
X_small = X[:N]
y_small = y[:N]

print("X_small shape:", X_small.shape)
print("y_small shape:", y_small.shape)


X_small shape: (8000, 30)
y_small shape: (8000,)


In [28]:
# Build mode and train
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = total_words
seq_len = X_small.shape[1]

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=32, input_length=seq_len))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

history = model.fit(X_small, y_small, epochs=2, batch_size=32, verbose=1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.0293 - loss: 7.5599
Epoch 2/2
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.0375 - loss: 6.0516


In [30]:
history = model.fit(X, y, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.0398 - loss: 5.9853
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.0445 - loss: 5.8847
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.0562 - loss: 5.8088
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.0571 - loss: 5.7388
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.0543 - loss: 5.6760
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.0674 - loss: 5.5796
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0687 - loss: 5.5322
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.0738 - loss: 5.4391
Epoch 9/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.0872 - loss: 5.3348
Epoch 10/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.0896 - loss: 5.2870


In [31]:
# Increase model capacity
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=64))  # bigger embedding
model.add(LSTM(128))  # more units
model.add(Dense(total_words, activation='softmax'))
max_len = 50  # gives more context



In [34]:
# Text Generation Function

def generate_text_temp(seed_text, next_words=30, temperature=1.0):
    output_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([output_text])[0]
        token_list = pad_sequences([token_list], maxlen=X.shape[1], padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_index = np.random.choice(len(preds), p=preds)
        next_word = tokenizer.index_word.get(next_index, "")
        output_text += " " + next_word
    return output_text

print(generate_text_temp("emma was", 50, temperature=0.7))


emma was pay churchill west scramble penitence therefore parts hasten panic relation become that joined absented over wisely wept groundless appears geography heat unpardonable clearing knight hasten flying clover explained demurs drunk passion demand understood indulgent slap convince delicacy struggled assuring paused bad eighteen and flirtation envy diverting dating impulse predominated mode


Notes:

Small dataset (~9970 sequences) and lightweight LSTM (64 units, embedding 32) was enough for demonstration purposes.


Text shows basic word-level coherence, though grammar isn’t perfect (expected in small toy models).

Temperature sampling as a technique to improve diversity.

# Task 3 Application Demonstration: Content Generation with LSTM
1️ Description (for report)

The trained LSTM model can generate new text sequences based on a seed phrase. This demonstrates a content creation application, where the model predicts one word at a time to continue a text in the style of Jane Austen.

Steps:

1 The user provides a seed text (e.g., "emma woodhouse was").

2 The model predicts the next word using the learned probabilities from the training data.

3 The predicted word is appended to the seed text.

4 Steps 2–3 are repeated for a specified number of words (next_words).

5 The output is a new, generated paragraph that follows the style and vocabulary of the original text.

This can be used for:

Generating creative writing content.

Extending stories or chapters automatically.

Text augmentation for NLP tasks.

In [35]:
# Implementation
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_text(seed_text, next_words, model, tokenizer, max_seq_len):
    """
    Generate text using the trained LSTM model.

    Parameters:
    seed_text (str) : initial text to start generation
    next_words (int) : number of words to generate
    model : trained LSTM model
    tokenizer : fitted Keras tokenizer
    max_seq_len (int) : sequence length used during training

    Returns:
    str : generated text
    """
    output_text = seed_text
    for _ in range(next_words):
        # Convert seed_text to sequence of token IDs
        token_list = [tokenizer.word_index[w] for w in seed_text.lower().split() if w in tokenizer.word_index]

        # Pad sequence
        token_list = pad_sequences([token_list], maxlen=max_seq_len, padding='pre')

        # Predict next word
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]

        # Find corresponding word
        next_word = [word for word, index in tokenizer.word_index.items() if index == predicted][0]

        # Append to text
        seed_text += " " + next_word
        output_text += " " + next_word

    return output_text

# -------------------------------
# Example usage
# -------------------------------
seed = "emma woodhouse was"
generated_text = generate_text(seed_text=seed, next_words=20, model=model, tokenizer=tokenizer, max_seq_len=max_seq_len-1)
print("Generated Text:\n", generated_text)


Generated Text:
 emma woodhouse was vacancies vacancies energy luxury iii parish prosings distresses ill resemblance wright cavil crayons consistently basis death advances advances looks sweetness
